In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
PATH = 'data/'

In [89]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [3]:
def get_numpy_data(df,features,output):
    df['constant'] = 1
    features = ['constant'] + features
    features_frame = df[features]
    features_matrix = features_frame.to_numpy()
    output_series = df[output]
    output_array = output_series.to_numpy()
    return features_matrix,output_array

In [4]:
def predict_output(features_matrix,weights):
    predictions = np.dot(features_matrix,weights)
    return predictions

In [6]:
def normalize_features(features_matrix):
    norms = np.linalg.norm(features_matrix,axis=0)
    normalized_features = features_matrix / norms
    return normalized_features,norms

In [10]:
sales = pd.read_csv(f'{PATH}kc_house_data.csv',dtype=dtype_dict)

In [12]:
features_matrix,output_array = get_numpy_data(sales, ['sqft_living','bedrooms'],'price')

In [15]:
normalized_features,norms = normalize_features(features_matrix)

In [16]:
initial_weights = np.array([1.,4.,1.])

In [18]:
predictions = predict_output(normalized_features,initial_weights)

In [30]:
ro = np.zeros(len(initial_weights))
for j in range(len(initial_weights)):
    ro[j] = (normalized_features[:,j]*(output_array-predictions+initial_weights[j]*normalized_features[:,j])).sum()

In [34]:
"{:e}".format(ro[1])

'8.793947e+07'

In [35]:
"{:e}".format(ro[2])

'8.096670e+07'

In [36]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i]*(output-prediction+weights[i]*feature_matrix[:,i])).sum()
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [40]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


In [60]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    max_change = None
    weights = initial_weights
    while max_change is None or max_change > tolerance:
        max_change = None
        for j in range(len(initial_weights)):
            original_weight = weights[j]
            weights[j] = lasso_coordinate_descent_step(j, feature_matrix, output, weights, l1_penalty)
            change = abs(original_weight-weights[j])
            if max_change is None or change > max_change:
                max_change = change
    return weights

In [46]:
features_matrix,output_array = get_numpy_data(sales, ['sqft_living','bedrooms'],'price')

In [47]:
normalized_features,norms = normalize_features(features_matrix)

In [62]:
coordinate_descent_weights = lasso_cyclical_coordinate_descent(normalized_features, output_array,np.zeros(3),1e7,1.0)

In [65]:
coordinate_descent_weights

array([21624997.95951909, 63157247.20788956,        0.        ])

In [67]:
def rss(x,y): return ((x-y)**2).sum()

In [69]:
"{:e}".format(rss(predict_output(normalized_features,coordinate_descent_weights),output_array))

'1.630492e+15'

In [90]:
train = pd.read_csv(f'{PATH}kc_house_train_data.csv', dtype=dtype_dict)
test = pd.read_csv(f'{PATH}kc_house_test_data.csv', dtype=dtype_dict)

In [91]:
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']

In [92]:
train_features_matrix,train_output = get_numpy_data(train,features,'price')

In [95]:
normalized_train_features,train_norms = normalize_features(train_features_matrix)

In [99]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_train_features,train_output,np.zeros(14),1e7,1)

In [108]:
weights1e7

array([24429600.23440312,        0.        ,        0.        ,
       48389174.77154896,        0.        ,        0.        ,
        3317511.21492165,  7329961.81171425,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [106]:
for i in weights1e7.nonzero()[0]:
    print(features[i-1])

yr_renovated
sqft_living
waterfront
view


In [107]:
weights1e8 = lasso_cyclical_coordinate_descent(normalized_train_features,train_output,np.zeros(14),1e8,1)

In [109]:
weights1e8

array([71114625.71488702,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [110]:
weights1e4 = lasso_cyclical_coordinate_descent(normalized_train_features,train_output,np.zeros(14),1e4,5e5)

In [111]:
weights1e4

array([ 78564738.34156762, -22097398.92430532,  12791071.87278517,
        93808088.09281193,  -2013172.75704954,  -4219184.93265014,
         6482842.81753506,   7127408.53480689,   5001664.8546964 ,
        14327518.43714051, -15770959.15237397,  -5159591.22213147,
       -84495341.7684364 ,   2824439.49703683])

In [112]:
normalized_weights1e7 = weights1e7/train_norms

In [113]:
normalized_weights1e7

array([1.85285530e+05, 0.00000000e+00, 0.00000000e+00, 1.61317458e+02,
       0.00000000e+00, 0.00000000e+00, 2.87664705e+05, 6.91937041e+04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00])

In [115]:
normalized_weights1e8 = weights1e8/train_norms
normalized_weights1e8

array([539366.62793373,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ])

In [116]:
normalized_weights1e4 = weights1e4/train_norms
normalized_weights1e4

array([ 5.95871771e+05, -4.80336244e+04,  4.30892643e+04,  3.12732803e+02,
       -3.46078585e-01, -2.01432664e+04,  5.62133764e+05,  6.72816325e+04,
        1.09255888e+04,  1.40325598e+04, -6.07214159e+01, -7.35796867e+01,
       -3.25079490e+02,  5.26011603e+01])

In [119]:
test_features_matrix,test_output = get_numpy_data(test,features,'price')
print("{:e}".format(rss(predict_output(test_features_matrix,normalized_weights1e7),test_output)))

2.759621e+14


In [120]:
print("{:e}".format(rss(predict_output(test_features_matrix,normalized_weights1e8),test_output)))

5.371662e+14


In [121]:
print("{:e}".format(rss(predict_output(test_features_matrix,normalized_weights1e4),test_output)))

2.284600e+14
